## MAKİNE ÖĞRENMESİNE GİRİŞ
### DOĞRUSAL REGRESYON MODELİ- LASSO REGRESYON


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("/Users/bernaersari/Downloads/Hitters.csv")

In [7]:
df # beyzbol oyuncularının çeşitli özeLliklerine göre ücretini belirleyen bir model kurmak istiyoruz.

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [8]:
df=df.dropna() #boş değerleri silelim.

In [9]:
df=df.dropna() #boş değerleri silelim.

In [10]:
df.select_dtypes(include='object').columns

Index(['League', 'Division', 'NewLeague'], dtype='object')

In [11]:
dms=pd.get_dummies(df[['League', 'Division', 'NewLeague']]) # one-hot encoding ile kategorik değişkenleri 0-1 değişkenlere dönüştürelim.

In [12]:
y=df['Salary'] #bulmak istediğimiz hedef değişkenimiz olan 'Salary' bağımlı değişkenimizi y olarak tanımlayalım.

In [13]:
X_=df.drop(['Salary','League', 'Division', 'NewLeague'], axis=1) 
#dataframeden encoding yapılan değişkenleri ve bağımlı değişkeni silip dataframedeki verileri X_ olarak tanımlayalım.

In [14]:
X_ #bağımsız değişkenlere göz atalım.

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12


In [15]:
X=pd.concat([X_,dms[['League_N', 'Division_W','NewLeague_N']]],axis=1) 
#son olarak tüm bağımsız değişkenleri bir araya getirelim. 
#Dummy değişkenler aynı bilgileri içerdiğinden her birinden bir tane değişkeni bağımsız değişken grubuna ekliyoruz.

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.25,
                                               random_state=42) # veri setinin yüzde 25 test seti olarak ayrıldı.


In [17]:
lasso_model=Lasso(alpha=0.5).fit(X_train,y_train)
#lasso regresyon modeli oluşturup,modeli eğitelim. 
#Alpha değeri ise, lasso regresyon modelinin matematiksel gösterimdeki karşılığı olan lambda değeridir.

In [18]:
lasso_model

Lasso(alpha=0.5)

In [19]:
lasso_model.coef_ #modeli eğittiğimizde alpha=0.5 olduğunda bulunan değişkenlere uygulanan katsayılar

array([-1.73393877e+00,  8.57550630e+00,  6.80899789e+00, -3.10353369e+00,
       -1.96811000e+00,  5.34478174e+00,  8.65651698e+00, -1.66689883e-01,
       -6.76272691e-02, -3.71757250e-01,  1.71037248e+00,  6.47397972e-01,
       -6.54025778e-01,  2.59969944e-01,  2.76190980e-01, -5.25405746e-01,
        1.00564401e+02, -9.81892031e+01, -3.62080261e+01])

In [20]:
pd.Series(lasso_model.coef_,index=X_train.columns).sort_values(ascending=False)
#hangi değişkenlere hangi katsayılar uygulanmış bir bakalım. 
#Burada hangi değişkenlerin beyzbol oyuncusunun maaşına etki ettiği görülüyor. 
#Örneğin, years değişkeninin maaşa olumlu katkısı olurken;errors değişkeninin maaşa etkisi negatif olduğu görülüyor.

League_N       100.564401
Years            8.656517
Hits             8.575506
HmRun            6.808998
Walks            5.344782
CRuns            1.710372
CRBI             0.647398
Assists          0.276191
PutOuts          0.259970
CHits           -0.067627
CAtBat          -0.166690
CHmRun          -0.371757
Errors          -0.525406
CWalks          -0.654026
AtBat           -1.733939
RBI             -1.968110
Runs            -3.103534
NewLeague_N    -36.208026
Division_W     -98.189203
dtype: float64

In [21]:
lasso_model.intercept_ #sabit katsayı

-6.799191196207516

In [22]:
lambda_numbers=np.sqrt(10**np.linspace(10,-1,50))
lambda_numbers

array([1.00000000e+05, 7.72244995e+04, 5.96362332e+04, 4.60537826e+04,
       3.55648031e+04, 2.74647411e+04, 2.12095089e+04, 1.63789371e+04,
       1.26485522e+04, 9.76778110e+03, 7.54312006e+03, 5.82513671e+03,
       4.49843267e+03, 3.47389211e+03, 2.68269580e+03, 2.07169840e+03,
       1.59985872e+03, 1.23548289e+03, 9.54095476e+02, 7.36795456e+02,
       5.68986603e+02, 4.39397056e+02, 3.39322177e+02, 2.62039853e+02,
       2.02358965e+02, 1.56270698e+02, 1.20679264e+02, 9.31939576e+01,
       7.19685673e+01, 5.55773659e+01, 4.29193426e+01, 3.31442475e+01,
       2.55954792e+01, 1.97659807e+01, 1.52641797e+01, 1.17876863e+01,
       9.10298178e+00, 7.02973212e+00, 5.42867544e+00, 4.19226744e+00,
       3.23745754e+00, 2.50011038e+00, 1.93069773e+00, 1.49097166e+00,
       1.15139540e+00, 8.89159334e-01, 6.86648845e-01, 5.30261134e-01,
       4.09491506e-01, 3.16227766e-01])

In [23]:
y_pred=lasso_model.predict(X_test) # test için ayırdığımız veri setini modelimiz tahmin yapsın.

In [24]:
hata1=np.sqrt(mean_squared_error(y_test,y_pred)) #Optimizasyon yapılmadan tahmin edilen değerlerimizin hatasına bakalım. 

#### MODEL TUNING


OPTİMUM LAMBDA DEĞERİNİ BULMAK İÇİN LASSOCV KULLANALIM.

In [25]:
lasso_model_cv=LassoCV(alphas=lambda_numbers,cv=10, max_iter=100000).fit(X_train,y_train)

In [26]:
lasso_model_cv.alpha_

202.35896477251575

In [27]:
#tune edilmiş final modelimizi kuralım.

In [28]:
lasso_model_final=Lasso(alpha=lasso_model_cv.alpha_).fit(X_train,y_train) #seçilmiş en iyi alpha değeri ile modeli eğitelim.

In [29]:
y_pred=lasso_model.predict(X_test) #modelimiz test veri seri üzerinden tahminleme yapsın.

In [30]:
hata=np.sqrt(mean_squared_error(y_test,y_pred)) #yapılan tahmnin hatasına bakalım.

In [31]:
hata

356.4431962995553

In [32]:
# ilk başta alpha=0.5 ile aldığımız hata değeri ile alpha değerinin 202.35 olarak seçiliip eğitilen modelin hatası
#aynı çıktı. Bu bazen gözlenen bir durumdur. Bu noktada farklı optimizasyon seöenekleri değerlendirilmeli,
#değişkenler gözden geçirilmelidir.

In [34]:
r2 = r2_score(y_test, y_pred)

In [35]:
r2 # bağımsız değişkenlerin bağımlı değişkeni açıklama gücüdür. Açıklama yüzdesidir. 

0.41309039627388355

In [36]:
# Bu değeri arttırmak için dğeişkenler ile ilgili bazı işlemler yapılabilir. 
#Veri setine ile eklemeler yapılabilir.